This is an intro / tutorial notebook for getting started with GTFS data.  

It is based on work that is shown on my dev blog [simplistic.me](http://simplistic.me/tag/gtfs.html). Posts there are not updated to current versions, mine or other packages'. Still, do check it out and feel free to comment or post an issue or pull request.

This is only one basic workflow which I like. You can find many more examples for working with GTFS online. There are tools and packages in many languages.

THE place to start is [awesome-transit](https://github.com/CUTR-at-USF/awesome-transit). 

TODO: Add more tools to this notebook - peartree, GTFSTK, UrbanAccess, Pandana

## Installation
TODO: Go to open-bus README (this needs to be there)

1. Install Anaconda3
1. Create a virtual environment (call it openbus or something indicative): `conda create -n openbus`
1. Clone [open-bus-explore](https://github.com/cjer/open-bus-explore) - `git clone https://github.com/cjer/open-bus-explore`
1. Install everything in [requirements.txt](https://github.com/cjer/open-bus-explore/blob/master/requirements.txt)
    1. partridge, peartree and GTFSTK require `pip install`
    1. the rest you should install using `conda install -c conda-forge <package_name>`
1. Run JupyterLab or Jupyter Notebook

## Imports and config

In [2]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import pandas as pd
import partridge as ptg
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import datetime

from gtfs_utils import *

alt.renderers.enable('notebook')
alt.data_transformers.enable('json')

sns.set_style("white")
sns.set_context("talk")
sns.set_palette('Set2', 10)

## Creating a `partridge` feed
We have a util function for getting a `partridge` feed object by date.  

In [4]:
LOCAL_GTFS_ZIP_PATH = 'data/gtfs_feeds/2019-02-03.zip' 
LOCAL_TARIFF_PATH = 'data/archive/2019-02-03/Tariff.zip' 

In [5]:
feed = get_partridge_feed_by_date(LOCAL_GTFS_ZIP_PATH, datetime.date.today())
type(feed)

partridge.gtfs.feed

* *Another option would be to use `ptg.get_representative_feed()` which finds the busiest day of the gtfs file and returns a feed for that day. Not showing this here.*

The feed has in it all the (standard) files in the original GTFS zip, as [pandas](https://github.com/pandas-dev/pandas) DataFrames.

In [6]:
[x for x in dir(feed) if not x.startswith('_')]

['agency',
 'calendar',
 'calendar_dates',
 'config',
 'fare_attributes',
 'fare_rules',
 'feed_info',
 'frequencies',
 'get',
 'is_dir',
 'path',
 'read_file_chunks',
 'routes',
 'shapes',
 'stop_times',
 'stops',
 'transfers',
 'trips',
 'view',
 'zmap']

In [24]:
feed.stops.stop_code.astype(int).max()


43078

In [16]:
feed.stops[feed.stops.stop_code.isin((feed.stops.groupby('stop_code').size()[feed.stops.groupby('stop_code').size()>1]).index)].sort_values('stop_code')

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,zone_id
24068,37404,10030,ת. מרכזית אשדוד/רציפים,רחוב:קניון הסיטי עיר: אשדוד רציף: 1 קומה:,31.790888,34.639905,0,11947,70
24071,37407,10030,ת. מרכזית אשדוד/רציפים,רחוב:קניון הסיטי עיר: אשדוד רציף: 5 קומה:,31.790888,34.639905,0,11947,70
24070,37406,10030,ת. מרכזית אשדוד/רציפים,רחוב:קניון הסיטי עיר: אשדוד רציף: 4 קומה:,31.790888,34.639905,0,11947,70
24069,37405,10030,ת. מרכזית אשדוד/רציפים,רחוב:קניון הסיטי עיר: אשדוד רציף: 3 קומה:,31.790888,34.639905,0,11947,70
24001,37265,10030,ת. מרכזית אשדוד/רציפים,רחוב:קניון הסיטי עיר: אשדוד רציף: 2 קומה:,31.790888,34.639905,0,11947,70
24072,37408,10030,ת. מרכזית אשדוד/רציפים,רחוב:קניון הסיטי עיר: אשדוד רציף: 6 קומה:,31.790888,34.639905,0,11947,70
23785,36918,10313,ת. מרכזית ערד/רציפים,רחוב: עיר: ערד רציף: 5 קומה:,31.255802,35.211836,0,12209,2560
23784,36917,10313,ת. מרכזית ערד/רציפים,רחוב: עיר: ערד רציף: 2 קומה:,31.255802,35.211836,0,12209,2560
23770,36877,10313,ת. מרכזית ערד/רציפים,רחוב: עיר: ערד רציף: 1 קומה:,31.255802,35.211836,0,12209,2560
23632,36154,10313,ת. מרכזית ערד/רציפים,רחוב: עיר: ערד רציף: 4 קומה:,31.255802,35.211836,0,12209,2560


Figuring out geographical zones requires using another zip file on MoT's FTP, using `get_zones_df()`. Which returns a simple mapping `stop_code` -> (Hebrew) `zone_name` in a DataFrame as well.

In [54]:
zones = get_zones_df(LOCAL_TARIFF_PATH)
zones.head()

,stop_code,zone_name
0,1,אזור בית שמש
1,2,אזור בית שמש
2,3,הרי ירושלים
3,4,אזור בית שמש
4,5,אזור בית שמש


In [78]:
feed.routes.shape

(6690, 7)

In [55]:
feed.routes.route_color.value_counts()

FF9933    1370
33CC33     228
9933FF       9
Name: route_color, dtype: int64

## Tidy DataFrame
A (monstrous) merged DataFrame for fancy analysis can be got using `get_tidy_feed_df()`, whom you pass a partridge feed and extra dataframes you want to merge to it (only `zones` is used here).

This takes a few minutes (MoT's GTFS is big)

In [56]:
f = get_tidy_feed_df(feed, [zones])

and what you get is this:

In [57]:
f.head()

,trip_id,departure_time,arrival_time,stop_id,stop_sequence,stop_name,stop_lat,stop_lon,stop_code,route_id,direction_id,route_short_name,route_long_name,agency_id,agency_name,zone_name
0,10096398_020219,21:00:00,21:00:00,34657,1,קניון קרני שומרון,32.174395,35.091190,63436,9735,1,70,קניון קרני שומרון-קרני שומרון<->אוניברסיטת ארי...,25,אפיקים,השומרון
1,10096398_020219,21:00:40,21:00:40,35317,2,שדרות רחבעם/קרני שומרון,32.175203,35.093854,65268,9735,1,70,קניון קרני שומרון-קרני שומרון<->אוניברסיטת ארי...,25,אפיקים,השומרון
2,10096398_020219,21:01:56,21:01:56,34436,3,קוממיות/שדרות רחבעם,32.175484,35.099323,63200,9735,1,70,קניון קרני שומרון-קרני שומרון<->אוניברסיטת ארי...,25,אפיקים,השומרון
3,10096398_020219,21:04:18,21:04:18,34444,4,נווה מנחם,32.179561,35.107716,63213,9735,1,70,קניון קרני שומרון-קרני שומרון<->אוניברסיטת ארי...,25,אפיקים,השומרון
4,10096398_020219,21:14:43,21:14:43,35506,5,יקיר בסיס צבאי,32.145644,35.116951,65290,9735,1,70,קניון קרני שומרון-קרני שומרון<->אוניברסיטת ארי...,25,אפיקים,השומרון


In the future I intend to make this more customizable (field selection, transformations and more). 

In [58]:
f.shape

(3388973, 16)

In [109]:
f.dtypes

trip_id                      object
departure_time      timedelta64[ns]
arrival_time        timedelta64[ns]
stop_id                      object
stop_sequence                 int64
stop_name                    object
stop_lat                    float64
stop_lon                    float64
stop_code                    object
route_id                   category
direction_id               category
route_short_name           category
route_long_name              object
agency_id                  category
agency_name                category
zone_name                  category
dtype: object

In [60]:
feed.stop_times.shape

(3388973, 8)

In [82]:
feed.routes[feed.routes.route_id=='10001']

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_color
2923,10001,14,91,האר''י/ישראל ב''ק-צפת<->שפרינצק/קרן היסוד-צפת-2#,27091-2-#,3,FF9933


In [91]:
school_routes = feed.routes[feed.routes.route_color=='FF9933'].route_id.values
school_routes

array(['270', '396', '397', ..., '25264', '25270', '25271'], dtype=object)

In [98]:
school_trips = f[f.route_id.isin(school_routes)].groupby('route_id').trip_id.nunique()
school_trips = school_trips[school_trips>0]

In [104]:
school_trips.mean()

1.5605839416058394

In [86]:
f.groupby('route_id').trip_id.nunique().mean()

14.3

In [106]:
school_trips.sum()

2138

In [107]:
f.trip_id.nunique()

95667

So we truly have all the stop times for one whole day of trips.

In [114]:
morning_school_trips = f[(f.route_id.isin(school_routes)) & (f.departure_time>=pd.Timedelta('7 hours')) & (f.departure_time<=pd.Timedelta('9 hours'))].groupby('route_id').trip_id.nunique()
morning_school_trips = morning_school_trips[morning_school_trips>0]

In [115]:
morning_school_trips.sum()

851

In [116]:
f[(f.departure_time>=pd.Timedelta('7 hours')) & (f.departure_time<=pd.Timedelta('9 hours'))].trip_id.nunique()

18176

In [124]:
morn_school_by_zone = f[(f.route_id.isin(school_routes)) & (f.departure_time>=pd.Timedelta('7 hours')) & (f.departure_time<=pd.Timedelta('9 hours'))].groupby('zone_name').trip_id.nunique()
morn_total_by_zone = f[(f.departure_time>=pd.Timedelta('7 hours')) & (f.departure_time<=pd.Timedelta('9 hours'))].groupby('zone_name').trip_id.nunique()

morn_zone = pd.concat([morn_school_by_zone, morn_total_by_zone], axis=1, keys=['school', 'total'])
morn_zone['ratio'] = morn_zone.school / morn_zone.total
morn_zone.sort_values('ratio', ascending=False)

,school,total,ratio
zone_name,,,
גליל עליון ורמת הגולן,57,340,0.167647
סובב כנרת ודרום רמת הגולן,34,317,0.107256
יוקנעם- טבעון,35,338,0.103550
כרמיאל,49,479,0.102296
זכרון,13,137,0.094891
נהריה,24,266,0.090226
השומרון,9,113,0.079646
נצרת,69,904,0.076327
חבל מודיעין,35,527,0.066414


In [125]:
morn_zone.shape

(39, 3)

In [129]:
morn_zone[morn_zone.ratio<0.01].shape

(17, 3)

In [130]:
morn_zone[morn_zone.ratio==0].shape

(12, 3)

וכדי לקבל סדר גודל. זה מתוך סה"כ 6690 חלופות ברחבי הארץ. קצת יותר מעשרים אחוז.

מן הסתם, אם מסתכלים על סך הנסיעות, זה הרבה פחות - קצת יותר מ-2%. אבל אם מסתכלים על נסיעות הבוקר בלבד (7 עד 9) הסעות תלמידים הן כמעט 5% מסך הנסיעות ברחבי הארץ.

בחתך אשכולות גם עולים דברים מעניינים. השיא הוא של גליל עליון ורמת הגולן - 17% מנסיעות הבוקר שם הן נסיעות תלמידים. במטרופולינים גוש דן, ירושלים וחיפה זה 1%, 4% ו-6% בהתאמה. יש גם לא מעט אשכולות ללא הסעות תלמידים בכלל. 17 מתוך 39 עם פחות מ-1%, 12 מתוכם ללא נסיעות תלמידים בכלל. הגדולים בינהם הם חדרה, רהט-להבים, נתיבות-שדרות ואופקים.

In [61]:
feed.stops.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,zone_id
0,1,38831,בי''ס בר לב/בן יהודה,רחוב:בן יהודה 76 עיר: כפר סבא רציף: קומה:,32.183939,34.917812,0,NaN,6900
1,2,38832,הרצל/צומת בילו,רחוב:הרצל עיר: קרית עקרון רציף: קומה:,31.870034,34.819541,0,NaN,469
2,3,38833,הנחשול/הדייגים,רחוב:הנחשול 30 עיר: ראשון לציון רציף: קומה:,31.984553,34.782828,0,NaN,8300
3,4,38834,פריד/ששת הימים,רחוב:משה פריד 9 עיר: רחובות רציף: קומה:,31.888550,34.790904,0,NaN,8400
4,6,38836,ת. מרכזית לוד/הורדה,רחוב: עיר: לוד רציף: קומה:,31.956576,34.898125,0,NaN,7000


In [73]:
fields = { 'street': 'רחוב', 
           'city': 'עיר', 
           'platform': 'רציף', 
           'floor': 'קומה' }

prefix='stop_desc_'

STOP_DESC_RE = ''
for n, fld in fields.items():
    STOP_DESC_RE += fld+f':(?P<{prefix+n}>.*)'
    
STOP_DESC_RE

'רחוב:(?P<stop_desc_street>.*)עיר:(?P<stop_desc_city>.*)רציף:(?P<stop_desc_platform>.*)קומה:(?P<stop_desc_floor>.*)'

In [74]:
#STOP_DESC_RE = 'רחוב:(.*)עיר:(.*)רציף:(.*)קומה:(.*)'

In [75]:
feed.stops.stop_desc.iat[0]

'רחוב:בן יהודה 76 עיר: כפר סבא רציף:   קומה:'

In [76]:
sd = feed.stops.stop_desc.str.extract(STOP_DESC_RE).apply(lambda x: x.str.strip())

sd.head()

,stop_desc_street,stop_desc_city,stop_desc_platform,stop_desc_floor
0,בן יהודה 76,כפר סבא,,
1,הרצל,קרית עקרון,,
2,הנחשול 30,ראשון לציון,,
3,משה פריד 9,רחובות,,
4,,לוד,,


In [77]:
pd.concat([feed.stops.stop_desc, sd], axis=1)

,stop_desc,stop_desc_street,stop_desc_city,stop_desc_platform,stop_desc_floor
0,רחוב:בן יהודה 76 עיר: כפר סבא רציף: קומה:,בן יהודה 76,כפר סבא,,
1,רחוב:הרצל עיר: קרית עקרון רציף: קומה:,הרצל,קרית עקרון,,
2,רחוב:הנחשול 30 עיר: ראשון לציון רציף: קומה:,הנחשול 30,ראשון לציון,,
3,רחוב:משה פריד 9 עיר: רחובות רציף: קומה:,משה פריד 9,רחובות,,
4,רחוב: עיר: לוד רציף: קומה:,,לוד,,
5,רחוב:חנה אברך 9 עיר: רחובות רציף: קומה:,חנה אברך 9,רחובות,,
6,רחוב:הרצל 20 עיר: קרית עקרון רציף: קומה:,הרצל 20,קרית עקרון,,
7,רחוב:הבנים 4 עיר: קרית עקרון רציף: קומה:,הבנים 4,קרית עקרון,,
8,רחוב:וייצמן 11 עיר: קרית עקרון רציף: קומה:,וייצמן 11,קרית עקרון,,
9,רחוב:האירוס 16 עיר: קרית עקרון רציף: קומה:,האירוס 16,קרית עקרון,,
